In [176]:
library(glmnet)
library(doMC)
library(survival)
library(data.table)
library(mltools)
library(CoxBoost)
library(randomForestSRC)
library(CoxHD)
source('run_prognosis.R')

In [20]:
df_all <-read.table("df_prognosis.tsv",sep = '\t' , header = T) 

###    2094 rows 166 columns

df_all <-na.omit(df_all) # delete rows with na (161)
df_all <- df_all[df_all$os>0,] # delete when os is negative (2)

####

#Convert predicted_component to one hot encoder
df_all$new_eln<-factor(df_all$new_eln, levels = c("adverse","intermediate","favorable"), labels = 0:2, ordered = TRUE)  # convert categorical new_eln to numerical (0,1,2)
name <-rownames(df_all)
df_all$predicted_component <- as.factor(df_all$predicted_component)
df_final <- as.data.frame(one_hot(as.data.table(df_all),cols="predicted_component"))
rownames(df_final) <- name

#write.table(df_final,"df_prognosis_features_ready.tsv",sep='\t',quote=F)
####

###  1931 rows 180 columns

In [21]:
#dim(df_all)
#dim(df_final)
df_final <- read.table("df_prognosis_features_ready.tsv",sep = '\t' , header = T)
x <- data.matrix(df_final[,1:84])
y <- data.matrix(df_final[,c("os","os_status")])
predictors <- c(predictorGLM)
#predictors <- c(rep(list(predictorGLM),10),rep(list(predictorRF),10),predictorBoost,predictorRFX)
str_predictors <-c(rep("CoxMod",10),rep("RFS",10),"CoxBoost","RFX")
l_alpha <-seq(0.1,1,0.1)
l_ntree <- seq(100,1000,100)
launch_prognosis(x,y,predictors,str_predictors,l_alpha,l_ntree)

[1] "start CV"


CoxMod_0.1
0.6265973
0.6511334
0.6376802
0.6470114
0.6554482
0.6808461
0.6326469
0.6370511
0.6442483
0.6153510


In [185]:
x <- data.matrix(df_final[,1:84])
y <- data.matrix(df_final[,c("os","os_status")])
predictors <- c(rep(list(predictorGLM),3),predictorRF,predictorRF,predictorBoost)
#predictors <- c(predictorRFX)
#predictors <- c(rep(list(predictorGLM),10),rep(list(predictorRF),10),predictorBoost,predictorRFX)
str_predictors <-c(rep("CoxMod",10),rep("RFS",10),"CoxBoost","RFX")
str_predictors <- c(rep("CoxGLM",3),rep("predictorRF",4),"predictorBoost")
l_alpha <-seq(0.05,1,0.05)
#l_ntree <- seq(100,1500,100)
l_ntree <-c(10,20)
(launch_prognosis(x=x,y=y,predictors=predictors,str_predictors=str_predictors,l_alpha=l_alpha,l_ntree=l_ntree,mc.cores=1,nodesize=c(10,20),nrepeats=1,nfolds=2))

[1] "start CV"
..[1] "start CV"
..[1] "start CV"
..[1] "start CV"
.[1] 10
.[1] 10
[1] "start CV"
.[1] 20
.[1] 20
[1] "start CV"
.[1] 10
.[1] 10
[1] "start CV"
.[1] 20
.[1] 20
[1] "start CV"
..

CoxGLM_0.05,CoxGLM_0.1,CoxGLM_0.15,predictorRF_10_10,predictorRF_10_20,predictorRF_20_10,predictorRF_20_20,predictorBoost_
0.6478436,0.6479093,0.6471204,0.6398329,0.6387570,0.6347908,0.6448320,0.6471647
0.6498326,0.6485409,0.6485937,0.6363847,0.6407016,0.6454629,0.6515815,0.6474822


In [255]:
#colnames(df_final)
all_features <- 1:177
eln_clin<-c(1,169:175)
eln_clin_demo<-c(1,169:177)
eln_clin_demo_cyto <-c(1,85:153,169:177)
eln_clin_demo_gen <-c(1:84,169:177)
eln_clin_demo_cyto_gen <-c(1:153,169:177)
eln_clin_demo_comp <-c(1,154:177)
eln_cyto_gen<-c(1:153)
eln_cyto_gen_comp <-c(1:168)
eln_cyto_comp <-c(1,85:168)
eln_gen_comp <-c(1:84,154:168)

clin_demo<-c(169:177)
clin_demo_cyto <-c(85:153,169:177)
clin_demo_gen <-c(2:84,169:177)
clin_demo_cyto_gen <-c(2:153,169:177)
clin_demo_comp <-c(154:177)
cyto_gen<-c(2:153)
cyto_gen_comp <-c(2:168)
cyto_comp <-c(85:168)
gen_comp <-c(2:84,154:168)
clin_demo_cyto_gen_comp<-c(2:177)
gen<-c(2:84)
cyto<-c(85:153)
comp<-c(154:168)
a <-list(all_features=all_features,eln_clin=eln_clin,eln_clin_demo=eln_clin_demo,
    eln_clin_demo_cyto=eln_clin_demo_cyto,eln_clin_demo_gen=eln_clin_demo_gen,
    eln_clin_demo_cyto_gen=eln_clin_demo_cyto_gen,eln_clin_demo_comp=eln_clin_demo_comp,
    eln_cyto_gen=eln_cyto_gen,eln_cyto_gen_comp=eln_cyto_gen_comp,eln_cyto_comp=eln_cyto_comp,
    eln_gen_comp=eln_gen_comp,clin_demo=clin_demo,clin_demo_cyto=clin_demo_cyto,clin_demo_gen=clin_demo_gen,
    clin_demo_cyto_gen=clin_demo_cyto_gen,clin_demo_comp=clin_demo_comp,cyto_gen=cyto_gen,cyto_gen_comp=cyto_gen_comp,
    cyto_comp=cyto_comp,gen_comp=gen_comp,clin_demo_cyto_gen_comp=clin_demo_cyto_gen_comp,gen=gen,cyto=cyto,comp=comp)

In [258]:

#predictors <- c(predictorBoost,predictorRFX)
prognosis_features<- list(comp=comp,clin_demo=clin_demo)
str_predictors <-c(rep("RFS",24))
l_alpha <-seq(0.1,1,0.1)
l_ntree <- seq(5,15,2)
l_ntree <- seq(5,6,1)
mc.cores <- 32
nodesize <- c(1,2,3,4)
nodesize <- c(1,2)
print ('q')
for (i in 1:length(prognosis_features)){
    print("DONE")
    x <- data.matrix(df_final[,prognosis_features[[i]]])
        launch_prognosis(x=x,y=y,predictors=predictors,str_predictors=str_predictors,l_alpha=l_alpha,
                l_ntree=l_ntree,mc.cores=mc.cores,nodesize=nodesize)
    print("DONE")
    }


[1] "q"
[1] "DONE"
[1] "start CV"
[1] "start CV"
[1] "DONE"
[1] "DONE"
[1] "start CV"
[1] "start CV"
[1] "DONE"


In [181]:
df_final <- read.table("df_prognosis_features_ready.tsv",sep = '\t' , header = T)
x <- data.matrix(df_final[,1:84])
y <- data.matrix(df_final[,c("os","os_status")])
predictors <- c(rep(list(predictorGLM),20),rep(list(predictorRF),105),predictorBoost,predictorRFX)
str_predictors <-c(rep("CoxGLM",20),rep("RFS",105),"CoxBoost","RFX")
l_alpha <-seq(0.05,1,0.05)
l_ntree <- seq(100,1500,100)
#write.table(launch_prognosis(x,y,predictors,str_predictors,l_alpha,l_ntree),"tests.tsv",quote=F,sep='\t')
launch_prognosis(x=x,y=y,predictors=predictors,str_predictors=str_predictors,l_alpha=l_alpha,l_ntree=l_ntree,mc.cores=8,nodesize=c(5,10,15,20,30,40,50))

In [266]:
summary(read.table("all_features.tsv",sep="\t"))

   CoxGLM_0.1       CoxGLM_0.2       CoxGLM_0.3       CoxGLM_0.4    
 Min.   :0.6904   Min.   :0.6913   Min.   :0.6917   Min.   :0.6915  
 1st Qu.:0.7030   1st Qu.:0.7054   1st Qu.:0.7062   1st Qu.:0.7065  
 Median :0.7106   Median :0.7135   Median :0.7149   Median :0.7147  
 Mean   :0.7120   Mean   :0.7135   Mean   :0.7143   Mean   :0.7145  
 3rd Qu.:0.7187   3rd Qu.:0.7214   3rd Qu.:0.7224   3rd Qu.:0.7225  
 Max.   :0.7357   Max.   :0.7371   Max.   :0.7389   Max.   :0.7395  
   CoxGLM_0.5       CoxGLM_0.6       CoxGLM_0.7       CoxGLM_0.8    
 Min.   :0.6923   Min.   :0.6922   Min.   :0.6921   Min.   :0.6920  
 1st Qu.:0.7069   1st Qu.:0.7072   1st Qu.:0.7073   1st Qu.:0.7074  
 Median :0.7155   Median :0.7149   Median :0.7148   Median :0.7150  
 Mean   :0.7147   Mean   :0.7146   Mean   :0.7147   Mean   :0.7147  
 3rd Qu.:0.7225   3rd Qu.:0.7230   3rd Qu.:0.7231   3rd Qu.:0.7231  
 Max.   :0.7401   Max.   :0.7403   Max.   :0.7409   Max.   :0.7411  
   CoxGLM_0.9        CoxGLM_1     

In [248]:
seq(500,1500,200)

[1]  500  700  900 1100 1300 1500

In [256]:
seq(5,7,1)

[1] 5 6 7